In [ ]:
'''
【项目12】  中国城市资本流动问题探索

数据：全国2013-2016所有企业间的投融资信息数据

作业要求
1、查看全国城际控股型投资关系
要求：
① 通过“data.xlsx”导出csv后，直接通过gephi看全国投资情况，有什么发现？
② 分别筛选出“同城投资”、“跨城投资”的TOP20，比较一下两类投资的数据分布
   ** 按照2013-2016年的汇总数据来计算
   ** 分开比较2013-2016四个年度的数据
   ** 需要绘制柱状图来辅助分析，这里用matplotlib即可
提示：
① 原始数据中，同一年中的投资数据会重复记录，所以需要将数据以'投资方所在城市','融资方所在城市','年份'这三个字段做一个分组汇总
② 用df.plot(kind = 'bar')来绘制图表，这里index为城市名即可

2、2013-2016年全国跨城市资本流动情况
要求：
① 结合“中国城市代码对照表.xlsx”数据，给2013-2016年“跨城投资”的汇总数据添加城市的经纬度
② 通过2013-2016年“跨城投资”的汇总数据，在gephi中绘制“城市关系图”
   ** 这里gephi中“点标签”只显示投资笔数TOP20的城市
③ 通过2013-2016年“跨城投资”的汇总数据，在echarts中绘制“全国跨城市资本流动OD图”
   ** 这里通过qgis的插件，做点转线
   ** 通过echart制作最后的资本
   ** 这里line的value为投资笔数
提示：
① 添加经纬度时，“投资方所在城市”、“融资方所在城市”都需要匹配城市经纬度
   ** 需要将“区/县”中最后一个字去掉
② gephi制图所需要导出的csv参考课程资料里的模板：边文件 → edge_model.csv；点文件 → node_model.csv（主要注意列名）
   ** 其中边数据的value需要做标准化处理 → 0-1之间
   ** gephi中单独显示某些城市的点标签方法 → 导入数据后，设置label，然后用label来显示标签，其中label只标注top20的数据
③ qgis中需要安装插件“LinePlotter”来转线
④ shapefile转geojson时：注意shapefile保持wgs84地理坐标系

3、深挖跨城市资本流动：钱从哪里来，到哪里去？
要求：
① 近四年对外控股型投资笔数最多的10个城市是哪些？
② 近四年吸引对外控股型投资笔数最多的10个城市又是哪些？
③ 从2013年到2016年，资本流动两大阵营的变化趋势：“北上深阵营”、“本地化阵营”
   ** “北上深阵营”：最大的外来投资方为北上深之一的城市
   ** “本地化阵营”：这里简化计算，将非“北上深阵营”都划入“本地化阵营”
   ** 该问题从“北上深阵营”历年来城市数量占比来看
   ** 可以qgis辅助绘制城市空间分布
提示：
① 资本流动两大阵营在计算中，主要以“融资方所在城市”为对象研究
② 资本流动两大阵营变化趋势计算中，可以构建函数，以年份为参数
③ 如何得到某年融资城市对应的最大的外来投资城市？
   ** 首先按照“融资方所在城市”做groupby分组，计算“投资企业对数”的max，得到一个Series
   ** Series通过reset_index转化为dataframe
   ** 再通过和源数据merge，找到该融资城市的最大外来投资对应的“投资方城市”，这里merge中的参数 on = ['融资方所在城市','投资企业对数']
④ 为了在qgis中制图，需要给数据添加经纬度信息，这里只需要添加“融资方所在城市”的经纬度
⑤ 为了qgis更好识别阵营类型，数据“阵营”字段用数字表示：0代表“本地化阵营”，1代表“北上深阵营”
⑥ qgis中制图时，既不属于“本地化阵营”又不属于“北上深阵营”的城市，颜色填充和“本地化阵营”一样即可

'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块

from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource,HoverTool
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

Loading BokehJS ...